In [1]:
from IPython.display import display, HTML#, clear_output, Image, Audio

import sys
import os

import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date

# matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.sans-serif'] = "Cambria"
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['mathtext.fontset'] = "stix"
plt.rcParams['figure.facecolor'] = 'white'

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

# demand fingerprint module
sys.path.insert(0, "../../demandfingerprint/")
import pdhelper

sys.path.insert(0, "./modules/")
import temphelper
import solargen

#dir management
data_dir = './data/'

In [2]:
import pvlib
import pytz

print(pytz.country_timezones('JP'))
tz = pytz.timezone('Asia/Tokyo')

latitude, longitude, tz, altitude, name = 33.5833, 130.3833, 'Asia/Tokyo', 2.5, 'Fukuoka'
loc = pvlib.location.Location(latitude, longitude, tz, altitude, name)

['Asia/Tokyo']


In [3]:
#panel configuration

panel_config = {'surface_tilt': 30, 'surface_azimuth': 180, 'albedo': 0.2,
               'panel': 'Kyocera_Solar_KD205GX_LP__2008__E__',
               'inverter': 'ABB__MICRO_0_25_I_OUTD_US_208__208V_'}

# Kyocera 206 Watt Solar Panel 
# https://www.ecodirect.com/Kyocera-KD205GX-LP-205-Watt-16-Volt-p/kyocera-kd205gx-lp.htm

# ABB Micro 250W microinverter
# https://www.solaris-shop.com/abb-micro-0-25-i-outd-us-208-240-250w-microinverter/

In [4]:
# load temperature data
# uses the monthly demand weighted temperature
kyushu_mean_temp_src = pd.read_csv(data_dir+'kyushu_temp_solar_capacity_weighted.csv', index_col='datetime')
kyushu_mean_temp_src.index = pd.to_datetime(kyushu_mean_temp_src.index)
kyushu_mean_temp = kyushu_mean_temp_src.copy()
kyushu_mean_temp = pdhelper.date_range_localized(kyushu_mean_temp)
kyushu_mean_temp = kyushu_mean_temp['kyushu']

# load ghi data
kyushu_ghi = pd.read_csv(data_dir+'kyushu_ghi_solar_capacity_weighted_mean.csv', index_col = 'datetime')
kyushu_ghi.index = pd.to_datetime(kyushu_ghi.index)
kyushu_ghi = pdhelper.date_range_localized(kyushu_ghi)
kyushu_ghi_3w = kyushu_ghi.rolling(3).mean()

In [5]:
# generation_stats = dict()
# for y in range(1990,1992):
#     ghi_df = kyushu_ghi_3w[str(y)]
#     w_data = solargen.build_weather_data(ghi_df, kyushu_mean_temp, loc)
#     power_gen_wmean = solargen.weather_to_power(w_data, loc, panel_config)
#     total_gen = power_gen_wmean.sum().values.tolist()
#     generation_stats[y] = power_gen_wmean.sum().values.tolist()

In [6]:
generation_stats = dict()
for y in range(1990,2020):
    ghi_df = kyushu_ghi_3w[str(y)]
    w_data = solargen.build_weather_data(ghi_df, kyushu_mean_temp, loc)
    power_gen_wmean = solargen.weather_to_power(w_data, loc, panel_config)
    total_gen = power_gen_wmean.sum().values.tolist()
    generation_stats[y] = power_gen_wmean.sum().values.tolist()

In [7]:
generation = pd.DataFrame(generation_stats, index = ['sapm']).T
generation['sapm_lf'] = generation['sapm']/(208*24*365)
#generation['sd_lf'] = generation['sd']/(208*24*365)
generation

,sapm,sapm_lf
1990,230550.336062,0.126531
1991,217034.029536,0.119113
1992,228051.697743,0.125160
1993,214119.922667,0.117514
1994,243484.816544,0.133630
1995,244435.516651,0.134152
1996,235608.224840,0.129307
1997,247126.447395,0.135629
1998,229144.846784,0.125760
1999,229790.977309,0.126115


In [8]:
# with reference year -- same year but different weather conditions
# 2018 was chosen because this is the start of the curtailment
ref_year_index = kyushu_mean_temp['2018'].index
n_hours = len(ref_year_index)

solar_power_gen = dict()
years = [2014,1991,2016,2018,2013,2012]
for year in years:
    ghi_df = kyushu_ghi_3w[str(year)]
    weather_data = solargen.build_weather_data(ghi_df, kyushu_mean_temp, loc)
    solar_gen = solargen.weather_to_power(weather_data, loc, panel_config)
    solar_gen = solar_gen['sapm'].values[:n_hours]
    solar_gen = pd.DataFrame(solar_gen, index=ref_year_index, columns = ['solar'])
    solar_power_gen[year] = solar_gen.copy()
    

In [9]:
solar_gen = pdhelper.dfdictExtractColumn(solar_power_gen, 'solar')
solar_p_max_pu = solar_gen/208 #208 W panel was used in the calculation
solar_p_max_pu = solar_p_max_pu.applymap(lambda x: round(x,6))
solar_p_max_pu.index = solar_p_max_pu.index.tz_localize(None)
# save to pypsa folder
if False:
    save_dir = './pypsa/csv_folder/'
    solar_p_max_pu.to_csv(save_dir+'solar_weather_p_max_pu.csv')